In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter
import string

from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ASUS/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
data = pd.read_csv('overAll.csv')
data.head()

,brand,model,title
0,pixel,9 pro fold,Some quick pictures with my 9 Pro Fold
1,pixel,9 pro fold,Pixel 9 Pro Fold home screen pictures
2,pixel,9 pro fold,Tap and raise to wake stopped on outer screen
3,pixel,9 pro fold,Who's going to pre-order pixel 9 Fold
4,pixel,9 pro fold,Trying to add Gmail accounts to Blackberry Hub.


In [6]:
data.shape

(41343, 3)

In [7]:
phone_counts = data['model'].value_counts()
phone_counts

model
One M8                 5107
Galaxy Note 8          4701
G4                     3892
G                      3610
Galaxy Note 9          2998
                       ... 
ROG Phone                 6
Galaxy A9                 4
XR20                      4
Galaxy Note 10 Lite       2
Blade V9                  1
Name: count, Length: 80, dtype: int64

In [8]:
data['phone_count'] = data['model'].map(phone_counts)
data = data.sort_values(by = 'phone_count' , ascending = False).reset_index(drop = True)
data.head(20)

,brand,model,title,phone_count
0,htc,One M8,My HTC M8 turned itself off this morning and w...,5107
1,htc,One M8,a first!,5107
2,htc,One M8,Google Handwrite,5107
3,htc,One M8,Sending Short Videos MMS,5107
4,htc,One M8,Camera on HTC one m8 not functioning with cert...,5107
5,htc,One M8,Text message contacts,5107
6,htc,One M8,My pictures all turned out horrible! (blurry a...,5107
7,htc,One M8,App cache is saving images in Gallery,5107
8,htc,One M8,HTC One M8 - Regular Version vs. Harman/Kardon...,5107
9,htc,One M8,good news,5107


In [9]:
data = data.drop('phone_count',axis = 1)
data.head()

,brand,model,title
0,htc,One M8,My HTC M8 turned itself off this morning and w...
1,htc,One M8,a first!
2,htc,One M8,Google Handwrite
3,htc,One M8,Sending Short Videos MMS
4,htc,One M8,Camera on HTC one m8 not functioning with cert...


In [10]:
data['title'] = data['title'].str.lower()
# Tokenize and remove stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords_and_lower(text):
    # Tokenize text and remove stopwords
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

data['tokenize_title'] = data['title'].apply(remove_stopwords_and_lower)

# Sentiment Analysis using VADER
sia = SentimentIntensityAnalyzer()

# Function to calculate sentiment score
def get_sentiment_scores(text):
    return sia.polarity_scores(text)

data['sentiment_scores'] = data['title'].apply(get_sentiment_scores)

# Now, split the 'sentiment_scores' into separate columns for 'positive', 'negative', 'neutral', and 'compound'
data['positive'] = data['sentiment_scores'].apply(lambda score_dict: score_dict['pos'])
data['negative'] = data['sentiment_scores'].apply(lambda score_dict: score_dict['neg'])
data['neutral'] = data['sentiment_scores'].apply(lambda score_dict: score_dict['neu'])
data['compound'] = data['sentiment_scores'].apply(lambda score_dict: score_dict['compound'])

# Drop the 'sentiment_scores' column if you no longer need it
data.drop(columns=['sentiment_scores'], inplace=True)

In [11]:
data.head(20)

,brand,model,title,tokenize_title,positive,negative,neutral,compound
0,htc,One M8,my htc m8 turned itself off this morning and w...,"[htc, m8, turned, morning, turn, back, ?]",0.156,0.000,0.844,0.3400
1,htc,One M8,a first!,"[first, !]",0.000,0.000,1.000,0.0000
2,htc,One M8,google handwrite,"[google, handwrite]",0.000,0.000,1.000,0.0000
3,htc,One M8,sending short videos mms,"[sending, short, videos, mms]",0.000,0.000,1.000,0.0000
4,htc,One M8,camera on htc one m8 not functioning with cert...,"[camera, htc, one, m8, functioning, certain, a...",0.196,0.149,0.655,0.1496
5,htc,One M8,text message contacts,"[text, message, contacts]",0.000,0.000,1.000,0.0000
6,htc,One M8,my pictures all turned out horrible! (blurry a...,"[pictures, turned, horrible, !, (, blurry, ove...",0.000,0.322,0.678,-0.5848
7,htc,One M8,app cache is saving images in gallery,"[app, cache, saving, images, gallery]",0.000,0.000,1.000,0.0000
8,htc,One M8,htc one m8 - regular version vs. harman/kardon...,"[htc, one, m8, -, regular, version, vs., harma...",0.255,0.000,0.745,0.3400
9,htc,One M8,good news,"[good, news]",0.744,0.000,0.256,0.4404


In [12]:
punctuation = set(string.punctuation)
# Function to get sorted word counts for each phone model
def get_sorted_word_counts(group):
    # Flatten all tokenized titles into a single list for the phone model
    all_tokens = [token for tokens in group['tokenize_title'] for token in tokens]
    
    # Filter out stopwords and punctuation, and convert to lowercase
    filtered_tokens = [word.lower() for word in all_tokens if word.lower() not in stop_words and word not in punctuation]
    
    # Count the frequency of each word
    word_counts = Counter(filtered_tokens)
    
    # Sort by most common words (most repeated first)
    most_common_words = word_counts.most_common()
    
    # Convert to a dictionary for easy analysis
    return dict(most_common_words)

# Group by 'about' (phone model) and aggregate sorted word counts for each phone model
sorted_word_counts_per_model = data.groupby(['brand','model']).apply(get_sorted_word_counts).reset_index()

# Rename the columns for clarity
sorted_word_counts_per_model.columns = ['brand','phone_model', 'sorted_word_counts']

# Convert to a DataFrame for better visualization of most repeated words
sorted_word_counts_df = pd.DataFrame(sorted_word_counts_per_model)
#print(sorted_word_counts_df)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23340\1915871414.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_word_counts_per_model = data.groupby(['brand','model']).apply(get_sorted_word_counts).reset_index()


In [13]:
# Set display option to show full content of the column
pd.set_option('display.max_colwidth', None)
# View the resulting DataFrame with each phone model and their sorted word counts
sorted_word_counts_df.head()

,brand,phone_model,sorted_word_counts
0,asus,ROG Phone,"{'rog': 5, 'phone': 4, 'asus': 2, 'security': 1, 'updates': 1, 'unlocking': 1, 'bootloader': 1, 'phones': 1, 'gearbest': 1, 'legit': 1, 'anybody': 1, 'used': 1, '1st': 1, 'generation': 1, 'google': 1, 'fi': 1, 'experience': 1, 'android': 1, 'pie': 1}"
1,asus,ROG Phone II,"{'phone': 7, 'rog': 7, 'asus': 5, 'ii': 4, '2': 3, 'time': 2, 'note': 2, 'user': 2, 'bootloader': 2, 'india': 2, 'announcement': 2, 'rogphone': 2, 'android': 2, 'back': 2, 'hope': 2, 'long': 1, 'samsung': 1, 'first': 1, 'u.s.': 1, 'pre-order': 1, 'live': 1, 'us': 1, 'lte': 1, 'support': 1, 'limits': 1, 'unlocked': 1, 'ca': 1, 'n't': 1, 'get': 1, 'update': 1, 'thoughts': 1, 'possible': 1, 'work': 1, 'around': 1, 'verizon': 1, 'users': 1, 'launch': 1, 'official': 1, 'mkbhd': 1, 'ifa': 1, '2019': 1, 'drool': 1, 'worthy': 1, 'content': 1, 'photos': 1, 'taken': 1, 'side': 1, 'load': 1, '12': 1, '5': 1, 'cant': 1, 'open': 1, 'youtube': 1, 'torch': 1, 'camera': 1, 'keeps': 1, 'crashing': 1, '3': 1, 'question': 1, 'rgb': 1, 'led': 1, 'aura': 1, 'lighting': 1, 'bug': 1, 'like': 1, 'updates': 1, '10': 1, 'available': 1, ''s': 1, 'stock': 1, 'gcam': 1, 'hype': 1, 'instagram': 1, 'opening': 1, 'randomly': 1, 'apps': 1, 'auto': 1, 'updating': 1, 'looking': 1, 'dedicated': 1, 'music': 1, 'player': 1, 'galaxy': 1, '10+': 1, 'keep': 1, 'coming': 1, 'holding': 1, 'sigh': 1, 'really': 1, 'black': 1, 'friday': 1, 'sale': 1, 'unlock': 1, 'tool': 1, 'already': 1, 'released': 1, 'volte/wi-fi': 1, 'calling': 1, 'working': 1, 't-mobile': 1, ...}"
2,asus,ZenFone,"{'zenfone': 74, 'asus': 48, '5': 35, 'phone': 18, 'n't': 14, 'help': 12, 'android': 9, 'get': 8, '4': 8, 'screen': 6, 'back': 6, 'ca': 6, 'working': 5, '10': 5, 'root': 5, 'display': 4, 'call': 4, 'update': 4, 'lollipop': 4, 'turn': 4, 'charge': 4, 'data': 4, 'usb': 4, 'charging': 4, 'unfortunately': 4, 'stopped': 4, 'connect': 3, 'c': 3, 'fix': 3, 'problem': 3, 'disable': 3, 'zenphone': 3, 'randomly': 3, 'apps': 3, 'cant': 3, 'calls': 3, 'anymore': 3, 'video': 3, '6': 3, 'unable': 3, 'receive': 3, 'logo': 3, 'mobile': 3, 'recover': 3, 'fast': 3, 'launcher': 3, 'need': 3, 'google': 3, 'battery': 3, '5.0': 3, 'restore': 3, 'files': 3, 'panel': 3, 'coming': 2, 'pc': 2, 'transfer': 2, 'light': 2, 'go': 2, 'problems': 2, 'bought': 2, 'lite': 2, 'latest': 2, 'rooted': 2, ''s': 2, 'sound': 2, 'restart': 2, 'bad': 2, 'notification': 2, 'users': 2, 'app': 2, 'audio': 2, 'properly': 2, 'buy': 2, 'selfie': 2, 'new': 2, '2': 2, 'partition': 2, 'cover': 2, 'stuck': 2, 'firmware': 2, 'card': 2, 'mode': 2, 'device': 2, 'save': 2, 'bricked': 2, 'upgrade': 2, 'wechat': 2, 'compatible': 2, 'memory': 2, 'remove': 2, 'icon': 2, 'zenui': 2, 'power': 2, 'button': 2, '3': 2, 'crashing': 2, 'max': 2, '6gb': 2, 'variant': 2, 'say': 2, ...}"
3,asus,ZenFone 2,"{'zenfone': 197, '2': 194, 'asus': 104, 'phone': 41, 'n't': 40, 'update': 28, 'get': 22, 'screen': 22, 'battery': 21, 'help': 20, 'working': 20, 'zenfone2': 19, 'new': 18, 'ca': 16, 'android': 16, 'ze551ml': 15, 'app': 15, 'data': 14, 'sim': 13, 'call': 13, 'zf2': 13, 'camera': 12, 'issue': 11, 'google': 11, 'charge': 11, 'apps': 11, 'charging': 11, 'anyone': 10, '2e': 10, 'model': 9, '...': 9, 'review': 9, 'root': 8, 'getting': 8, 'mode': 8, ''s': 8, 'work': 8, 'connect': 8, 'connection': 8, 'zenphone': 8, 'today': 8, 'problems': 7, 'card': 7, 'text': 7, 'keyboard': 7, 'wo': 7, '``': 7, '''': 7, 'internet': 7, 'firmware': 7, 'problem': 7, 'issues': 7, 'us': 7, 'case': 7, 'turn': 6, 'notifications': 6, 'notification': 6, 't-mobile': 6, 'quick': 6, 'way': 6, 'fix': 6, 'laser': 6, 'first': 6, 'got': 6, 'question': 6, 'charger': 6, 'wifi': 6, 'message': 6, 'system': 6, 'lte': 6, 'sd': 5, 'mobile': 5, 'mms': 5, 'dual': 5, 'experience': 5, 'email': 5, 'drain': 5, 'receive': 5, 'messages': 5, 'vs': 5, 'stuck': 5, 'photo': 5, 'buy': 5, 'version': 5, 'moto': 5, 'download': 5, 'samsung': 5, 'pc': 5, 'little': 5, 'thoughts': 5, 'updates': 5, 'error'

In [14]:
data['model'].value_counts()

model
One M8                 5107
Galaxy Note 8          4701
G4                     3892
G                      3610
Galaxy Note 9          2998
                       ... 
ROG Phone                 6
Galaxy A9                 4
XR20                      4
Galaxy Note 10 Lite       2
Blade V9                  1
Name: count, Length: 80, dtype: int64

In [15]:
sorted_word_counts_df[sorted_word_counts_df['brand'] == 'samsung']

,brand,phone_model,sorted_word_counts
39,samsung,Galaxy A3,"{'a3': 120, 'samsung': 93, 'galaxy': 57, 'n't': 26, 'screen': 24, 'phone': 18, 'get': 14, 'fix': 11, 'turn': 10, 'camera': 9, 'wo': 8, 'ca': 8, 'android': 8, '2016': 8, 'contacts': 8, 'storage': 8, 'app': 7, 'working': 7, 'show': 7, 'notification': 7, 'problem': 7, '2017': 7, 'new': 6, 'stop': 6, 'apps': 6, 'update': 6, 'work': 5, 'sd': 5, 'marshmallow': 5, 'issue': 5, 'home': 5, 'call': 5, 'delete': 5, 'frozen': 5, 'planner': 4, 'card': 4, 'turned': 4, 'blank': 4, '6.0.1': 4, 'size': 4, 'internet': 4, 'help': 4, 'back': 4, 'sound': 4, 'know': 4, 'battery': 4, 'lock': 4, 'wifi': 4, 'audio': 3, 'bluetooth': 3, 'odin': 3, 'froze': 3, 'charging': 3, 'download': 3, 'best': 3, 'pics': 3, 'wrong': 3, 'usb': 3, 'missing': 3, 'upgrade': 3, 'unlock': 3, 'problems': 3, 'lte': 3, 'remove': 3, 'using': 3, 'advice': 3, 'google': 3, 'change': 3, '5.0.2': 3, 'works': 3, 'button': 3, 'device': 3, 'number': 3, 'play': 2, 'stuck': 2, 'mode': 2, 'buttons': 2, 'open': 2, 'time': 2, 'pictures': 2, 'showing': 2, 'arriving': 2, 'day': 2, 'playstore': 2, 'save': 2, 'saved': 2, 'flickering': 2, 'otg': 2, 'longer': 2, 'pc': 2, 'incoming': 2, 'email': 2, 'properly': 2, 'shows': 2, 'stays': 2, 'options': 2, 'outgoing': 2, 'emails': 2, 'iphone': 2, 'delay': 2, ...}"
40,samsung,Galaxy A5,"{'a5': 211, 'samsung': 132, 'galaxy': 107, '2016': 45, '2017': 41, 'n't': 38, 'phone': 31, 'screen': 29, 'working': 19, 'help': 19, 'ca': 17, 'update': 17, 'battery': 16, 'problem': 15, 'get': 13, 'sound': 12, 'notification': 11, 'new': 11, 'data': 11, 'apps': 11, 'turn': 11, 'wifi': 10, 'android': 10, 'wo': 10, 'download': 10, 'mode': 10, ''s': 10, 'app': 10, 'time': 10, 'issues': 9, 'showing': 9, 'notifications': 8, 'button': 8, 'charging': 8, 'change': 8, 'software': 8, 'icon': 8, '...': 8, 'sd': 8, 'storage': 8, 'root': 8, 'light': 8, 'lock': 7, 'work': 7, 'card': 7, 'nougat': 6, 'version': 6, 'need': 6, 'camera': 6, 'mobile': 6, 'keeps': 6, 'back': 6, 'drain': 6, 'keyboard': 6, 'reset': 6, 'appear': 6, 'contacts': 6, 'bluetooth': 6, 'fast': 6, 'stuck': 6, 'message': 6, 'connect': 6, 'display': 5, 'home': 5, 'speaker': 5, '``': 5, '''': 5, 'low': 5, 'unable': 5, 'issue': 5, 'cant': 5, 'please': 5, 'fix': 5, 'option': 5, 'open': 5, 'upgrade': 5, 'volume': 5, 'use': 5, 'connection': 5, 'set': 5, 'make': 4, 'install': 4, 'settings': 4, 'front': 4, 'facebook': 4, 'text': 4, 'package': 4, 'call': 4, 'usb': 4, 'otg': 4, 'every': 4, 'delete': 4, 'active': 4, 'want': 4, 'charge': 4, 'remove': 4, 'google': 4, 'photos': 4, 'model': 4, 'private': 4, ...}"
41,samsung,Galaxy A9,"{'a9': 3, 'galaxy': 2, 'samsung': 2, 'notification': 1, 'led': 1, 'software': 1, 'update': 1, 'option': 1, 'missing': 1, 'device': 1, 'buying': 1, 'change': 1, 'keyboard': 1}"
42,samsung,Galaxy Alpha,"{'alpha': 110, 'galaxy': 74, 'samsung': 44, 'n't': 21, 'battery': 16, 'phone': 16, 'get': 12, 'help': 12, 'update': 11, 'ca': 11, 'screen': 9, 'lollipop': 9, 'problem': 7, 'working': 6, 'anyone': 6, 'buy': 5, 'phones': 5, 'scanner': 5, 'root': 5, 'apps': 5, 'text': 5, 'call': 5, 'password': 5, 'g850f': 5, '...': 5, 'mode': 5, 'life': 4, 'turn': 4, 'time': 4, 'fingerprint': 4, 'change': 4, 'settings': 4, 'app': 4, 'camera': 4, 'data': 4, 'delete': 4, 'google': 4, 'extended': 4, 'charging': 4, 'open': 4, 'g850a': 4, 'new': 3, 'fix': 3, 'android': 3, '5.0.2': 3, 'sm-g850f': 3, 'device': 3, 'unable': 3, 'automatically': 3, 'messages': 3, 'upgrade': 3, 'iphone': 3, '2500': 3, 'mah': 3, 'wifi': 3, 'wireless': 3, 'back': 3, 'something': 3, 'else': 3, 'saving': 3, 'home': 3, 'set': 3, 'contacts': 3, 'wo': 3, 'mobile': 3, 'charge': 3, 'backup': 3, 'audio': 3, 'lock': 3, 'kitkat': 3, 'using': 3, 'make': 3, 'speaker': 3, 'recover': 3, ''ve': 2, 'stop': 2, 'notifications': 2, 'aplha': 2, 'showing': 2, 'wrong': 2, 'problems': 2, 'hibernation': 2, 'questions': 2, 'slow': 2, 'connection': 2, 'log': 2, 'unlock': 2, 'icon': 2, ''s': 2, 'normal': 2, 'rooting': 2, 'cus

In [16]:
sorted_word_counts_df[sorted_word_counts_df['phone_model'] == 'One M8']  

,brand,phone_model,sorted_word_counts
11,htc,One M8,"{'m8': 2321, 'htc': 2077, 'one': 1595, 'phone': 305, 'n't': 304, 'update': 272, 'screen': 259, 'get': 221, 'help': 205, 'camera': 203, 'battery': 199, 'app': 186, 'new': 171, 'verizon': 151, 'google': 137, '``': 136, '''': 136, 'working': 127, 'lollipop': 119, 'card': 115, 'issues': 113, 'ca': 106, 'issue': 105, '...': 104, 'apps': 102, 'sd': 101, 'android': 101, 'wifi': 100, 'problem': 98, 'sprint': 93, ''s': 90, 'wo': 88, 'anyone': 87, 'question': 81, 'mode': 78, 'notification': 78, '2': 76, 'back': 75, 'problems': 75, 'music': 73, 'fix': 69, 'text': 67, 'lock': 65, 'bluetooth': 64, 'case': 64, 'unlocked': 63, 'sync': 61, 'play': 61, 'data': 60, 'work': 60, 'marshmallow': 60, 'photos': 59, 'turn': 58, 'power': 58, 'notifications': 58, 'please': 57, 'charge': 56, 'email': 55, '....': 54, 'stuck': 54, 'contacts': 53, 'stock': 53, 'use': 52, 'life': 51, 'using': 50, 'stopped': 50, 'video': 49, 'need': 49, 'coming': 49, 'keyboard': 48, 't-mobile': 48, 'charging': 48, 'getting': 46, 'time': 46, 'make': 46, 'sense': 45, 'call': 44, 'messages': 44, 'via': 44, 'software': 42, 'sim': 42, 'best': 42, 'questions': 42, 'button': 42, 'come': 42, 'gallery': 41, 'see': 41, 'voice': 41, 'want': 41, 'volume': 40, 'sound': 40, 'storage': 38, 'stop': 38, 'reset': 38, 'view': 38, 'still': 37, 'got': 37, 'slow': 37, 'vs': 37, 'look': 37, ...}"
